In [1]:
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy einops 

In [2]:
import os
import time
import numpy as np
import pandas as pd
from typing import Dict, List

import matplotlib.pyplot as plt

import torch
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoModelForCausalLM,
    AutoTokenizer, 
    TrainingArguments, 
    Trainer,
    EarlyStoppingCallback
)

import warnings
warnings.filterwarnings("ignore")

In [3]:
os.environ['WANDB_DISABLED']="true"

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
print(device)

cuda


### Load model

In [6]:
model_name = 'Biscottezi/vit5-base-finetuned-vitext2sql'
tokenizer = AutoTokenizer.from_pretrained(model_name)
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
original_model = original_model.to(device)

tokenizer_config.json:   0%|          | 0.00/2.45k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/820k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/741 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/904M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [7]:
original_model

T5ForConditionalGeneration(
  (shared): Embedding(36096, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

### Load dataset

In [8]:
dataset_train = load_dataset("hieulhwork24/custom-vietnamese-text2sql", split='train[:90%]')
dataset_val = load_dataset("hieulhwork24/custom-vietnamese-text2sql", split='train[-10%:]')
dataset = DatasetDict({
    'train': dataset_train,
    'validation': dataset_val
})
dataset.save_to_disk("completed_train_dataset")
print("Created train dataset!")

README.md:   0%|          | 0.00/118 [00:00<?, ?B/s]

train_data.csv:   0%|          | 0.00/656k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/494 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/445 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/49 [00:00<?, ? examples/s]

Created train dataset!


In [9]:
dataset['train'][0]

{'id': 1,
 'context': '\nCó 1 bảng tên jidouka cần truy vấn. Bảng cần truy vấn bao gồm các cột: \nid: số thứ tự của hàng (int);\ninnovation_name: tên của tác phẩm cải tiến (str);\ntask_type: Tác phẩm cải tiến đó sinh ra để làm gì? (str) (ví dụ: Xử lí database, nhập thông tin, tối ưu quy trình làm việc,...) ;\ntool: Công cụ để thực hiện (str) (ví dụ: Python, Excel, Visual Studio Code, ...);\ndescribe_innovation: Mô tả rõ ràng hơn mục đích của công cụ (giải thích rõ hơn cột task_type) (str)  ;\nproduct: Output của công cụ có định dạng như thế nào (str) (ví dụ: file csv, file xlsx, ....);\npic: Tên người phụ trách quản lí công cụ (str)  ;\ndc: Phòng ban làm việc của người phụ trách quản lí công cụ (str) (dc1, dc2, dc3, dcd, souko,...);\nsaved_hours: số lượng giờ mà nhờ việc áp dụng cải tiến tiết kiệm được (int);\ncreated_at: Thời điểm công cụ này ra mắt (str) (ví dụ: 2024-10-11, 2024-10-09,...);\ninformation: Đường link youtube tài liệu hướng dẫn sử dụng công cụ (str)\n    ',
 'question':

### Preprocessing data

In [10]:
def tokenize_function(sample):
    """
    Convert dataset to instructions for LLM
    Args:
    sample: a record from dataset include id, context, questions, sql_answer
    """
    start_prompt = "Context:\n"
    middle_prompt = "\n\nQuestion:\n"
    end_prompt = "\n\nAnswer:\n"

    data_zip = zip(sample['context'], sample['question'])
    prompt = [start_prompt + context + middle_prompt + question + end_prompt for context, question in data_zip]
    sample['input_ids'] = tokenizer(prompt, padding=True, truncation=True, return_tensors="pt").input_ids
    sample['labels'] = tokenizer(sample['answer'], padding=True, truncation=True, return_tensors="pt").input_ids


    return sample

In [11]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'context', 'question','answer'])

Map:   0%|          | 0/445 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/49 [00:00<?, ? examples/s]

In [12]:
tokenized_datasets['train'][0]

{'input_ids': [12832,
  11199,
  35862,
  1457,
  40,
  2206,
  473,
  3006,
  342,
  1539,
  2647,
  913,
  713,
  833,
  35792,
  5609,
  913,
  713,
  833,
  845,
  825,
  72,
  3034,
  35862,
  8943,
  35862,
  210,
  739,
  487,
  54,
  312,
  33,
  3833,
  35831,
  35899,
  1611,
  15777,
  2394,
  35997,
  13792,
  35862,
  473,
  54,
  643,
  962,
  1573,
  772,
  33,
  6494,
  35831,
  35899,
  4,
  21079,
  35997,
  1951,
  4701,
  35862,
  4476,
  962,
  1573,
  772,
  173,
  457,
  170,
  180,
  239,
  1080,
  35933,
  33,
  6494,
  35831,
  33,
  35802,
  35850,
  2038,
  35862,
  5025,
  3741,
  35,
  371,
  27075,
  35790,
  930,
  326,
  345,
  35790,
  1229,
  2449,
  212,
  498,
  239,
  217,
  35790,
  5402,
  35831,
  207,
  1904,
  468,
  35862,
  321,
  1352,
  180,
  305,
  219,
  33,
  6494,
  35831,
  33,
  35802,
  35850,
  2038,
  35862,
  24044,
  35790,
  30441,
  35790,
  17770,
  11445,
  18782,
  35790,
  9255,
  35831,
  35899,
  23405,
  1177,
  4829,


### Test model before fine-tuning

In [13]:
question = dataset['validation'][0]['question']
context = dataset['validation'][0]['context']
answer = dataset['validation'][0]['answer']

prompt = f"""Context:
{context}

Question:
{question}

Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
inputs = inputs.to(device)

output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Context:

Có 1 bảng tên jidouka cần truy vấn. Bảng cần truy vấn bao gồm các cột: 
id: số thứ tự của hàng (int);
innovation_name: tên của tác phẩm cải tiến (str);
task_type: Tác phẩm cải tiến đó sinh ra để làm gì? (str) (ví dụ: Xử lí database, nhập thông tin, tối ưu quy trình làm việc,...) ;
tool: Công cụ để thực hiện (str) (ví dụ: Python, Excel, Visual Studio Code, ...);
describe_innovation: Mô tả rõ ràng hơn mục đích của công cụ (giải thích rõ hơn cột task_type) (str)  ;
product: Output của công cụ có định dạng như thế nào (str) (ví dụ: file csv, file xlsx, ....);
pic: Tên người phụ trách quản lí công cụ (str)  ;
dc: Phòng ban làm việc của người phụ trách quản lí công cụ (str) (dc1, dc2, dc3, dcd, souko,...);
saved_hours: số lượng giờ mà nhờ việc áp dụng cải tiến tiết kiệm được (int);
created_at: Thời điểm công cụ này ra mắt (str) (ví dụ: 2024-10-11, 2024-10-09,...);
infor

In [14]:
to_train = True
model_name = 'Biscottezi/vit5-base-finetuned-vitext2sql'
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
finetuned_model = finetuned_model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [30]:
# finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("finetuned_model_10_epoch")
# finetuned_model = finetuned_model.to(device)

In [15]:
%%time

if to_train:
    output_dir = f'/kaggle/working/sql-training-{str(int(time.time()))}'

    training_args = TrainingArguments(
        output_dir=output_dir,
        learning_rate=5e-3,
        num_train_epochs=100,
        per_device_train_batch_size=16,     # batch size per device during training
        per_device_eval_batch_size=16,      # batch size for evaluation
        weight_decay=0.01,
        logging_steps=50,
        evaluation_strategy='steps',        # evaluation strategy to adopt during training
        eval_steps=50,                     # number of steps between evaluation
    )

    early_stopping_callback = EarlyStoppingCallback( 
        early_stopping_patience=5
    )

    trainer = Trainer(
        model=finetuned_model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['validation'],
        callbacks=[early_stopping_callback]
    )
    
    trainer.train()
    
    finetuned_model.save_pretrained("finetuned_model_100_epoch")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
50,5.875000,2.603316
100,2.395300,2.156250
150,2.188100,1.859375
200,1.962500,1.567921
250,1.775900,1.442921
300,1.903400,2.402105
350,2.139100,1.614796
400,1.837800,1.458546
450,1.628400,1.273756
500,1.826700,2.613520


CPU times: user 49min 55s, sys: 16min 33s, total: 1h 6min 28s
Wall time: 48min 47s


In [16]:
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("finetuned_model_100_epoch")
finetuned_model = finetuned_model.to(device)

In [18]:
question = "Công cụ nào giúp tôi tiết kiệm thời gian khi làm việc?"
context = """Có 1 bảng cần truy vấn. 
Bảng cần truy vấn bao gồm các cột: 
id: số thứ tự của hàng (int);
innovation_name: tên của tác phẩm cải tiến (str);
task_type: Tác phẩm cải tiến đó sinh ra để làm gì? (str) (ví dụ: Xử lí database, nhập thông tin, tối ưu quy trình làm việc,...) ;
tool: Công cụ để thực hiện (str) (ví dụ: Python, Excel, Visual Studio Code, ...);
describe_innovation: Mô tả rõ ràng hơn mục đích của công cụ (giải thích rõ hơn cột task_type) (str)  ;
product: Output của công cụ có định dạng như thế nào (str) (ví dụ: file csv, file xlsx, ....);
pic: Tên người phụ trách quản lí công cụ (str)  ;
dc: Phòng ban làm việc của người phụ trách quản lí công cụ (str) (dc1, dc2, dc3, dcd, souko,...);
saved_hours: số lượng giờ mà nhờ việc áp dụng cải tiến tiết kiệm được (int);
created_at: Thời điểm công cụ này ra mắt (str) (ví dụ: 2024-10-11, 2024-10-10,...);
information: Đường link youtube tài liệu hướng dẫn sử dụng công cụ (str)"""
answer = """
SELECT innovation_name, saved_hours FROM jidouka ORDER BY saved_hours DESC LIMIT 1;"""
prompt = f"""Context:
{context}

Question:
{question}

Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
inputs = inputs.to(device)

output = tokenizer.decode(
    finetuned_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{answer}\n')
print(dash_line)
print(f'FINE-TUNED MODEL - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Context:
Có 1 bảng cần truy vấn. 
Bảng cần truy vấn bao gồm các cột: 
id: số thứ tự của hàng (int);
innovation_name: tên của tác phẩm cải tiến (str);
task_type: Tác phẩm cải tiến đó sinh ra để làm gì? (str) (ví dụ: Xử lí database, nhập thông tin, tối ưu quy trình làm việc,...) ;
tool: Công cụ để thực hiện (str) (ví dụ: Python, Excel, Visual Studio Code, ...);
describe_innovation: Mô tả rõ ràng hơn mục đích của công cụ (giải thích rõ hơn cột task_type) (str)  ;
product: Output của công cụ có định dạng như thế nào (str) (ví dụ: file csv, file xlsx, ....);
pic: Tên người phụ trách quản lí công cụ (str)  ;
dc: Phòng ban làm việc của người phụ trách quản lí công cụ (str) (dc1, dc2, dc3, dcd, souko,...);
saved_hours: số lượng giờ mà nhờ việc áp dụng cải tiến tiết kiệm được (int);
created_at: Thời điểm công cụ này ra mắt (str) (ví dụ: 2024-10-11, 2024-10-10,...);
information: Đườn